In [1]:
# Task 1 dls
class GoalBasedAgent:
    def __init__(self, goal):
        self.goal = goal

    def formulate_goal(self, percept):
        if percept == self.goal:
            return "Goal reached"
        return "Searching"

    def dls(self, graph, start, goal, depth_limit):
        visited = []

        def dfs(node, depth):
            if depth > depth_limit:
                return NotImplementedError
            visited.append(node)
            if node == goal:
                print(f"Goal found with DLS. Path: {visited}")
                return visited

            for neighbor in graph.get(node, []):
                if neighbor not in visited:
                    path = dfs(neighbor, depth + 1)
                    if path:
                        return path

            visited.pop()
            return None

        return dfs(start, 0)

    def act(self, percept, graph, depth):
        goal_status = self.formulate_goal(percept)
        if goal_status == "Goal reached":
            return f"Goal {self.goal} found!"
        else:
            path = self.dls(graph, percept, self.goal, depth)
            if path:
                return f"Path found: {path}"
            else:
                return "Goal not found within depth limit"


class Environment:
    def __init__(self, graph):
        self.graph = graph

    def get_percept(self, node):
        return node


def run_agent(agent, environment, start_node, depth):
    percept = environment.get_percept(start_node)
    action = agent.act(percept, environment.graph, depth)
    print(action)

tree = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F', 'G'],
    'D': ['H'],
    'E': [],
    'F': ['I'],
    'G': [],
    'H': [],
    'I': []
}

start_node = 'A'
goal_node = 'I'

agent = GoalBasedAgent(goal_node)
environment = Environment(tree)

run_agent(agent, environment, start_node, 3)




Goal found with DLS. Path: ['A', 'C', 'F', 'I']
Path found: ['A', 'C', 'F', 'I']


In [2]:
#Task 1 UCS
import random

graph = {
'A': {'B': 2, 'C': 1},
'B': {'D': 4, 'E': 3},
'C': {'F': 1, 'G': 5},
'D': {'H': 2},
'E': {},
'F': {'I': 6},
'G': {},
'H': {},
'I': {}
}
start = 'A'
goal = 'I'

class Environment:
  def __init__(self, graph, start, goal):
    self.graph = graph
    self.start = start
    self.goal = goal

class UtilityBasedAgent:

  def ucs(self, graph, start, goal):
      # Initialize the frontier with the start node and cost 0
      frontier = [(start, 0)] # (node, cost)
      visited = set() # Set to keep track of visited nodes
      cost_so_far = {start: 0} # Cost to reach each node
      came_from = {start: None} # Path reconstruction
      while frontier:
        # Sort frontier by cost, simulate priority queue
        frontier.sort(key=lambda x: x[1])
        # Pop the node with the lowest cost
        current_node, current_cost = frontier.pop(0)
        # If we've already visited this node, skip it
        if current_node in visited:
          continue

        # Mark the current node as visited
        visited.add(current_node)
        # If we reach the goal, reconstruct the path and return
        if current_node == goal:
          path = []
          while current_node is not None:
            path.append(current_node)
            current_node = came_from[current_node]
          path.reverse()
          print(f"Goal found with UCS. Path: {path}, Total Cost: {current_cost}")
          return

        # Explore neighbors
        for neighbor, cost in graph[current_node].items():
          new_cost = current_cost + cost
          if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
            cost_so_far[neighbor] = new_cost
            came_from[neighbor] = current_node
            frontier.append((neighbor, new_cost)) # Add to frontier

      print("Goal not found")


def run_agent(agent, environment):
  agent.ucs(environment.graph, environment.start, environment.goal)

agent = UtilityBasedAgent()
environment = Environment(graph, start, goal)

run_agent(agent, environment)

Goal found with UCS. Path: ['A', 'C', 'F', 'I'], Total Cost: 8


In [3]:
#Task 2
from itertools import permutations

def calculate_cost(graph, path):
    cost = 0
    for i in range(len(path) - 1):
        cost += graph[path[i]][path[i+1]]
    cost += graph[path[-1]][path[0]]
    return cost

def tsp(graph):
    nodes = list(graph.keys())
    min_cost = float('inf')
    best_path = []

    for start in nodes:
        remaining_nodes = [n for n in nodes if n != start]
        all_paths = permutations(remaining_nodes)

        print(f"\nStarting at {start}:")

        for path in all_paths:
            full_path = [start] + list(path)
            cost = calculate_cost(graph, full_path)
            print(f"Path: {' -> '.join(full_path)} -> {start}, Cost: {cost}")

            if cost < min_cost:
                min_cost = cost
                best_path = full_path

    return best_path, min_cost


graph = {
    '1': {'2': 10, '3': 15, '4': 20},
    '2': {'1': 10, '3': 35, '4': 25},
    '3': {'1': 15, '2': 35, '4': 30},
    '4': {'1': 20, '2': 25, '3': 30}
}

best_path, best_cost = tsp(graph)


print("\nBest Path:", " -> ".join(best_path) + f" -> {best_path[0]}")
print("Minimum Cost:", best_cost)



Starting at 1:
Path: 1 -> 2 -> 3 -> 4 -> 1, Cost: 95
Path: 1 -> 2 -> 4 -> 3 -> 1, Cost: 80
Path: 1 -> 3 -> 2 -> 4 -> 1, Cost: 95
Path: 1 -> 3 -> 4 -> 2 -> 1, Cost: 80
Path: 1 -> 4 -> 2 -> 3 -> 1, Cost: 95
Path: 1 -> 4 -> 3 -> 2 -> 1, Cost: 95

Starting at 2:
Path: 2 -> 1 -> 3 -> 4 -> 2, Cost: 80
Path: 2 -> 1 -> 4 -> 3 -> 2, Cost: 95
Path: 2 -> 3 -> 1 -> 4 -> 2, Cost: 95
Path: 2 -> 3 -> 4 -> 1 -> 2, Cost: 95
Path: 2 -> 4 -> 1 -> 3 -> 2, Cost: 95
Path: 2 -> 4 -> 3 -> 1 -> 2, Cost: 80

Starting at 3:
Path: 3 -> 1 -> 2 -> 4 -> 3, Cost: 80
Path: 3 -> 1 -> 4 -> 2 -> 3, Cost: 95
Path: 3 -> 2 -> 1 -> 4 -> 3, Cost: 95
Path: 3 -> 2 -> 4 -> 1 -> 3, Cost: 95
Path: 3 -> 4 -> 1 -> 2 -> 3, Cost: 95
Path: 3 -> 4 -> 2 -> 1 -> 3, Cost: 80

Starting at 4:
Path: 4 -> 1 -> 2 -> 3 -> 4, Cost: 95
Path: 4 -> 1 -> 3 -> 2 -> 4, Cost: 95
Path: 4 -> 2 -> 1 -> 3 -> 4, Cost: 80
Path: 4 -> 2 -> 3 -> 1 -> 4, Cost: 95
Path: 4 -> 3 -> 1 -> 2 -> 4, Cost: 80
Path: 4 -> 3 -> 2 -> 1 -> 4, Cost: 95

Best Path: 1 -> 2 -> 4 

In [14]:
#Task 3
def dls(graphOrTree, node, goal, depth, path):
  if depth == 0:
    return False
  if node == goal:
    path.append(node)
    return True
  if node not in graphOrTree:
    return False
  for child in graphOrTree[node]:
    if dls(graphOrTree, child, goal, depth - 1, path):
      path.append(node)
      return True
  return False

def iterative_deepening(graphOrTree, start, goal, max_depth):
  for depth in range(max_depth + 1):
    path = []
    if dls(graphOrTree, start, goal, depth, path):
      print(f'Solution found at depth {depth}')
      print("Path to goal:", " → ".join(reversed(path)))
      return
  print("Goal not found within depth limit.")

start_node = 'X'
goal_node = 'O'

tree = {
    'X': ['Y', 'Z'],
    'Y': ['P', 'Q'],
    'Z': ['R', 'S'],
    'P': ['M', 'N'],
    'Q': ['T'],
    'R': ['O', 'U'],
    'S': [],
    'M': [],
    'N': [],
    'O': [],
    'T': [],
    'U': []
}


graph = {
    'X': ['Y', 'Z', 'P'],
    'Y': ['P', 'Q', 'R', 'T'],
    'Z': ['R', 'S', 'X', 'U'],
    'P': ['M', 'O', 'Y'],
    'Q': ['Y', 'Z', 'P', 'R'],
    'R': ['M', 'O', 'S'],
    'S': ['Z', 'T'],
    'M': ['O', 'N'],
    'N': ['M'],
    'T': ['O'],
    'U': ['Z', 'O'],
    'O': ['M', 'U']
}

max_search_depth = 5
print("Using Tree:")
iterative_deepening(tree, start_node, goal_node, max_search_depth)

print("\n")
print("Using Graph:")
iterative_deepening(graph, start_node, goal_node, max_search_depth)

Using Tree:
Solution found at depth 4
Path to goal: X → Z → R → O


Using Graph:
Solution found at depth 3
Path to goal: X → P → O


In [12]:
#Task 4
from collections import deque


DIRECTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1)] # Up, Down, Left, Right

def find_position(grid, char):

    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == char:
                return (i, j)
    return None

def is_valid(grid, x, y, visited):

    return 0 <= x < len(grid) and 0 <= y < len(grid[0]) and grid[x][y] != 1 and (x, y) not in visited

def bfs(grid):
    start = find_position(grid, 'S')
    goal = find_position(grid, 'G')

    if not start or not goal:
        return "Start or Goal not found."

    queue = deque([(start, [start])])
    visited = set()
    visited.add(start)

    while queue:
        (x, y), path = queue.popleft()

        if (x, y) == goal:
            return path

        for dx, dy in DIRECTIONS:
            new_x, new_y = x + dx, y + dy

            if is_valid(grid, new_x, new_y, visited):
                queue.append(((new_x, new_y), path + [(new_x, new_y)]))
                visited.add((new_x, new_y))

    return "No path found."


grid = [
    [0, 0, 'S', 0, 0],
    [0, 1, 1, 0, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 'G', 1, 0],
    [0, 1, 0, 0, 0]
]


shortest_path = bfs(grid)


print("Shortest Path:", shortest_path)


Shortest Path: [(0, 2), (0, 3), (1, 3), (2, 3), (2, 2), (3, 2)]
